# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 27 2022 10:39AM,253823,15,ALI433931,"Alliance Pharma, Inc.",Released
1,Dec 27 2022 10:39AM,253823,15,ALI029357469,"Alliance Pharma, Inc.",Released
2,Dec 27 2022 10:39AM,253823,15,ALI038104889,"Alliance Pharma, Inc.",Released
3,Dec 27 2022 10:39AM,253823,15,ALI039366542,"Alliance Pharma, Inc.",Released
4,Dec 27 2022 10:39AM,253823,15,ALI063491309,"Alliance Pharma, Inc.",Released
5,Dec 27 2022 10:36AM,253821,10,Clear-8955476,"ClearSpec, LLC",Released
6,Dec 27 2022 10:33AM,253820,10,Enova-11141,Emerginnova,Released
7,Dec 27 2022 10:33AM,253820,10,Enova-11142-2,Emerginnova,Released
8,Dec 27 2022 10:33AM,253820,10,Eme-106920,Emerginnova,Released
9,Dec 27 2022 10:32AM,253819,10,Enova-11139,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,253818,Released,2
26,253819,Released,2
27,253820,Released,3
28,253821,Released,1
29,253823,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253818,NaN,2.0
253819,NaN,2.0
253820,NaN,3.0
253821,NaN,1.0
253823,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253783,0.0,1.0
253784,24.0,5.0
253788,0.0,1.0
253789,0.0,1.0
253790,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253783,0,1
253784,24,5
253788,0,1
253789,0,1
253790,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253783,0,1
1,253784,24,5
2,253788,0,1
3,253789,0,1
4,253790,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253783,,1
1,253784,24,5
2,253788,,1
3,253789,,1
4,253790,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc."
5,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC"
6,Dec 27 2022 10:33AM,253820,10,Emerginnova
9,Dec 27 2022 10:32AM,253819,10,Emerginnova
11,Dec 27 2022 10:16AM,253818,20,"Exact-Rx, Inc."
13,Dec 27 2022 10:15AM,253815,10,ISDIN Corporation
14,Dec 27 2022 10:15AM,253814,10,ISDIN Corporation
67,Dec 27 2022 10:15AM,253813,10,ISDIN Corporation
133,Dec 27 2022 10:15AM,253811,10,ISDIN Corporation
210,Dec 27 2022 10:14AM,253817,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc.",,5
1,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC",,1
2,Dec 27 2022 10:33AM,253820,10,Emerginnova,,3
3,Dec 27 2022 10:32AM,253819,10,Emerginnova,,2
4,Dec 27 2022 10:16AM,253818,20,"Exact-Rx, Inc.",,2
5,Dec 27 2022 10:15AM,253815,10,ISDIN Corporation,,1
6,Dec 27 2022 10:15AM,253814,10,ISDIN Corporation,,53
7,Dec 27 2022 10:15AM,253813,10,ISDIN Corporation,,66
8,Dec 27 2022 10:15AM,253811,10,ISDIN Corporation,,77
9,Dec 27 2022 10:14AM,253817,10,Eywa Pharma Inc.,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc.",5,
1,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC",1,
2,Dec 27 2022 10:33AM,253820,10,Emerginnova,3,
3,Dec 27 2022 10:32AM,253819,10,Emerginnova,2,
4,Dec 27 2022 10:16AM,253818,20,"Exact-Rx, Inc.",2,
5,Dec 27 2022 10:15AM,253815,10,ISDIN Corporation,1,
6,Dec 27 2022 10:15AM,253814,10,ISDIN Corporation,53,
7,Dec 27 2022 10:15AM,253813,10,ISDIN Corporation,66,
8,Dec 27 2022 10:15AM,253811,10,ISDIN Corporation,77,
9,Dec 27 2022 10:14AM,253817,10,Eywa Pharma Inc.,8,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc.",5,
1,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC",1,
2,Dec 27 2022 10:33AM,253820,10,Emerginnova,3,
3,Dec 27 2022 10:32AM,253819,10,Emerginnova,2,
4,Dec 27 2022 10:16AM,253818,20,"Exact-Rx, Inc.",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc.",5.0,NaN
1,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC",1.0,NaN
2,Dec 27 2022 10:33AM,253820,10,Emerginnova,3.0,NaN
3,Dec 27 2022 10:32AM,253819,10,Emerginnova,2.0,NaN
4,Dec 27 2022 10:16AM,253818,20,"Exact-Rx, Inc.",2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 10:39AM,253823,15,"Alliance Pharma, Inc.",5.0,0.0
1,Dec 27 2022 10:36AM,253821,10,"ClearSpec, LLC",1.0,0.0
2,Dec 27 2022 10:33AM,253820,10,Emerginnova,3.0,0.0
3,Dec 27 2022 10:32AM,253819,10,Emerginnova,2.0,0.0
4,Dec 27 2022 10:16AM,253818,20,"Exact-Rx, Inc.",2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5583724,421.0,0.0
12,253797,0.0,1.0
15,253823,5.0,0.0
16,253806,1.0,0.0
19,507588,2.0,0.0
20,507602,7.0,24.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5583724,421.0,0.0
1,12,253797,0.0,1.0
2,15,253823,5.0,0.0
3,16,253806,1.0,0.0
4,19,507588,2.0,0.0
5,20,507602,7.0,24.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,421.0,0.0
1,12,0.0,1.0
2,15,5.0,0.0
3,16,1.0,0.0
4,19,2.0,0.0
5,20,7.0,24.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,421.0
1,12,Released,0.0
2,15,Released,5.0
3,16,Released,1.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Executing,0.0,1.0,0.0,0.0,0.0,24.0
Released,421.0,0.0,5.0,1.0,2.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Executing,0.0,1.0,0.0,0.0,0.0,24.0
1,Released,421.0,0.0,5.0,1.0,2.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Executing,0.0,1.0,0.0,0.0,0.0,24.0
1,Released,421.0,0.0,5.0,1.0,2.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()